In [22]:
import pandas as pd
import time
import requests
import os
import zipfile
from os import listdir
from os.path import isfile, join
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn import metrics

In [2]:
response = requests.get('https://jazz-yt67mbf5eq-ey.a.run.app')
response.status_code

200

In [34]:
url = 'https://jazz-yt67mbf5eq-ey.a.run.app/implement_som'

counter = 1

while counter <= 5:

    files = {
    "jazz_file": open('Kohonen-Maps/Jazz_Files/data_dog_domain_file_'+str(counter)+'.jazz', 'rb'), 
    "csv_file": open('Kohonen-Maps/data_dog_prepared.csv','rb')
    }

    response = requests.post(url, files=files)

    filename = url.split('/')[-1] + '_result.zip'
    with open(os.path.join('Kohonen-Maps', filename), "wb") as output_file:
        output_file.write(response.content)

    with zipfile.ZipFile('Kohonen-Maps/implement_som_result.zip', 'r') as zip_ref:
        name = zip_ref.namelist()
        # extract the "dimension reduced" csv file
        zip_ref.extract(name[2])

    onlyfiles = [f for f in listdir('data/processed') if isfile(join('data/processed', f))]
    old_file = os.path.join('data//processed', onlyfiles[counter-1])
    new_file = os.path.join('data//processed', 'dimension_reduced_0'+str(counter)+'.csv')
    os.rename(old_file, new_file)

    counter += 1

<h1>Test-Training-Split for V6</h1>

In [35]:
df6 = pd.read_csv('data\processed\dimension_reduced_01.csv')
df6 = df6.drop('Unnamed: 0', axis=1)
df6.head()

,size_dog,eye_circularity,skull_circularity,ear_size,eyes_distance,ear_form,hair_tone,nose_elongatedness,weight_dog,muzzle_form,skin_tone_main,skin_pattern_complexity,body_length,tail_length,hair_curlness,hair_length,leg_front_number,leg_back_number,breuni_class
0,-0.213402,0.125494,0.168467,1.995971,4.155209,4.604628,0.112969,0.394827,-0.069599,0.106257,1.358165,-0.338491,0.281004,3.150630,-0.861860,0.454387,-0.392876,-0.101937,fashion_lifestyle_elite
1,-0.074873,-0.109234,0.111949,0.602378,1.455808,1.755093,0.454189,0.383406,0.266451,-0.168195,0.483018,-0.193844,-0.036693,1.157513,-0.522307,0.502820,-0.210901,-0.024823,['main_stream' 'trendy']
2,-0.021847,-0.160417,0.091410,0.262266,0.808839,1.071884,0.514274,0.346454,0.320689,-0.220955,0.271665,-0.159504,-0.104362,0.681765,-0.445645,0.527719,-0.170320,-0.006165,main_stream
3,-0.042043,-0.109373,0.103349,0.587222,1.456644,1.763536,0.441451,0.340466,0.239100,-0.158088,0.481030,-0.198670,-0.030152,1.164377,-0.542648,0.544200,-0.220114,-0.023942,trendy
4,-0.045733,-0.086194,0.108780,0.749253,1.789809,2.123597,0.409625,0.333485,0.197341,-0.127777,0.588389,-0.221276,0.006654,1.414919,-0.601018,0.567907,-0.249038,-0.032685,trendy


In [36]:
data = df6.values
X, y = data[:, :-1], data[:, -1]
print(X.shape, y.shape)
# using the train test split function
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(299, 18) (299,)
(209, 18) (90, 18) (209,) (90,)


In [37]:
# Random Forest Classifier:
model = RandomForestClassifier(random_state=1)
model.fit(X_train, y_train)
# make predictions
yhat = model.predict(X_test)
# evaluate predictions
acc = accuracy_score(y_test, yhat)
print('Accuracy: %.3f' % acc)

Accuracy: 0.567


In [38]:
# K Nearest Neighbor Classification:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
# make predictions
y_pred = knn.predict(X_test)
# evaluate predictions
print('Accuracy:', metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.4444444444444444
